# Delivery 3: Full Walkthrough


Akshata Sandeep Patki  

Mtk Nr:- 414540

In [18]:
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: ads_covid-19'

# Update the data from johns hopkins git repo and get current data from germany

In [19]:
import subprocess
import os
import sys

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    ''' Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure
    '''
    git_repo = 'https://github.com/CSSEGISandData/COVID-19.git'
    path = "data/raw/COVID-19/"
    git_pull = subprocess.Popen( "git clone "+ git_repo,
                         cwd = os.path.dirname( os.path.realpath(path)),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


def get_current_data_germany():
    ''' Get current data from germany, attention API endpoint not too stable
        Result data frame is stored as pd.DataFrame

    '''
    # 16 states
    #data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    # 400 regions / Landkreise
    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object=json.loads(data.content)
    full_list=[]
    for pos,each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])

    pd_full_list=pd.DataFrame(full_list)
    pd_full_list.to_csv('data/raw/NPGEO/GER_state_data.csv')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))

if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()

Error : b"Cloning into 'COVID-19'...\nUpdating files:  11% (235/1978)\rUpdating files:  12% (238/1978)\rUpdating files:  13% (258/1978)\rUpdating files:  14% (277/1978)\rUpdating files:  15% (297/1978)\rUpdating files:  16% (317/1978)\rUpdating files:  17% (337/1978)\rUpdating files:  18% (357/1978)\rUpdating files:  19% (376/1978)\rUpdating files:  19% (382/1978)\rUpdating files:  20% (396/1978)\rUpdating files:  21% (416/1978)\rUpdating files:  22% (436/1978)\rUpdating files:  23% (455/1978)\rUpdating files:  24% (475/1978)\rUpdating files:  25% (495/1978)\rUpdating files:  26% (515/1978)\rUpdating files:  27% (535/1978)\rUpdating files:  28% (554/1978)\rUpdating files:  28% (568/1978)\rUpdating files:  29% (574/1978)\rUpdating files:  30% (594/1978)\rUpdating files:  31% (614/1978)\rUpdating files:  32% (633/1978)\rUpdating files:  33% (653/1978)\rUpdating files:  34% (673/1978)\rUpdating files:  35% (693/1978)\rUpdating files:  36% (713/1978)\rUpdating files:  36% (715/1978)\rUpdat

# Required Data set preparation

In [20]:
import pandas as pd
import numpy as np

from datetime import datetime

def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set

    '''

data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)
pd_data_base=pd_raw.rename(columns={'Country/Region':'Country',
                      'Province/State':'State'})

pd_data_base['State']=pd_data_base['State'].fillna('no')

pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
pd_relational_model=pd_data_base.set_index(['State','Country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'Confirmed'},
                                                  )
pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')
pd_relational_model.to_csv('../ads_covid-19/data/processed/COVID_relational_confirmed.csv',index=False)


## Analyzing data for Country US as it is in a new file due to the new structure of John Hopkins

data_path_US='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
pd_raw_US=pd.read_csv(data_path_US)

pd_raw_US=pd_raw_US.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','FIPS', 'Admin2',  'Lat', 'Long_', 'Combined_Key'],axis=1)
pd_data_base_US=pd_raw_US.rename(columns={'Province_State':'State'}).copy()
pd_relation_model_US=pd_data_base_US.set_index(['State']).T
pd_relation_model_US=pd_relation_model_US.stack().reset_index().rename(columns={'level_0':'date',0:'Confirmed'})
pd_relation_model_US['Country']='US'
pd_relation_model_US['date']=[datetime.strptime( each,"%m/%d/%y") for each in pd_relation_model_US.date]

##Swap the columns according to the prvious stored data in pd_relational_model

columns_titles = ["date","State","Country","Confirmed"]
pd_relation_model_US=pd_relation_model_US.reindex(columns=columns_titles)

## Merge the US data
pd_relational_model_all=pd_relational_model[pd_relational_model['Country']!='US'].reset_index(drop=True)
pd_relational_model_all=pd.concat([pd_relational_model_all,pd_relation_model_US],ignore_index=True)

## Save the complete data
pd_relational_model_all.to_csv('data/processed/20220719_COVID_relational_confirmed.csv',index=False)

print(' Number of rows stored: '+str(pd_relational_model_all.shape[0]))
print(' Latest date is: '+str(max(pd_relational_model_all.date)))


if __name__ == '__main__':

    store_relational_JH_data()


 Number of rows stored: 3317790
 Latest date is: 2022-07-24 00:00:00


# Slope Calculation

In [21]:
# %load src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='Confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='Confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='Confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['State','Country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['State','Country',filter_on]].groupby(['State','Country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='Confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['State','Country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['State','Country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('data/processed/20220719_COVID_relational_confirmed.csv',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'Confirmed_filtered')


    mask=pd_result_larg['Confirmed']>100
    pd_result_larg['Confirmed_filtered_DR']=pd_result_larg['Confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('../ads_covid-19/data/processed/COVID_final_set.csv',index=False)
    print(pd_result_larg[pd_result_larg['Country']=='Germany'].tail())

the test slope is: [2.]
              date State  Country   Confirmed  Confirmed_filtered  \
3198835 2022-07-20    no  Germany  30131303.0          30109983.0   
3198836 2022-07-21    no  Germany  30239122.0          30205473.6   
3198837 2022-07-22    no  Germany  30331131.0          30272764.4   
3198838 2022-07-23    no  Germany  30331133.0          30321931.5   
3198839 2022-07-24    no  Germany  30331133.0          30371098.6   

         Confirmed_DR  Confirmed_filtered_DR  
3198835  2.160716e+02             253.222339  
3198836  2.464517e+02             269.804637  
3198837  3.025988e+02             371.001523  
3198838  6.586269e+02             519.788235  
3198839  3.033113e+07             616.711815  


# Data Visualization

In [22]:
# %load src/visualization/visualize.py
import pandas as pd
import numpy as np

import dash
dash.__version__
#import dash_core_components as dcc
from dash import dcc
#import dash_html_components as html
from dash import html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())

df_input_large=pd.read_csv('data/processed/COVID_final_set.csv')


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Statistical Analysis of COVID-19 data using Data Science

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['Country'].unique()],
        value=['India', 'Germany','Italy'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'Confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'Confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'Confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'Confirmed_filtered_DR'},
    ],
    value='Confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'DR' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (#stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['Country']==each]

        if show_doubling=='Confirmed_filtered_DR':
            df_plot=df_plot[['State','Country','Confirmed','Confirmed_filtered','Confirmed_DR','Confirmed_filtered_DR','date']].groupby(['Country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['State','Country','Confirmed','Confirmed_filtered','Confirmed_DR','Confirmed_filtered_DR','date']].groupby(['Country','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                line_width=1,
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)

C:\Users\aksha\ads_covid-19
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
